In [ ]:
get_ipython().run_cell_magic('capture', '', '!pip install db-dtypes\n!pip install keras\n!pip install tensorflow\n')

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', 'import pandas as pd\nimport numpy as np\nimport os\nfrom google.cloud import bigquery\nfrom google.oauth2 import service_account\nfrom google.cloud.bigquery import magics\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.svm import SVR\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.linear_model import BayesianRidge\nfrom sklearn.linear_model import ElasticNetCV\nfrom keras.models import Sequential\nfrom keras.layers import Bidirectional, LSTM, Dropout, Dense\n')

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query_main = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
query_job = bigquery_client.query(query_main)
unemployment_data = query_job.to_dataframe()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [ ]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()

In [ ]:
query_pred = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
query_job = bigquery_client.query(query_pred)
prediction_list = query_job.to_dataframe()

In [ ]:
print(unemployment_data.info())
print(wage_data.info())

In [ ]:
# check shape of both frames to see if they are joinable
print('Unemployment df size:', unemployment_data.shape)
print('Wage df size:', wage_data.shape)

In [ ]:
unemployment_data.isnull().sum() / len(unemployment_data) * 100

In [ ]:
wage_data.isnull().sum() / len(wage_data) * 100

In [ ]:
# replace values with 0
clean_unemploymentDf = unemployment_data.copy()
clean_unemploymentDf.fillna(0, inplace=True)
clean_unemploymentDf

In [ ]:
clean_unemploymentDf.isnull().sum() #check

In [ ]:
# check correlation
correlation = clean_unemploymentDf.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
plt.figure(figsize=(15,10))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f')

In [ ]:
# check wage information
wage_data[wage_data['average_wage'].isnull()] # there's 3 nulls here - might as well drop them and use this tract to attempt to join the datasets together; or impute with mean

In [ ]:
#wage_data.dropna(axis=0, inplace=True)
wage_data['average_wage'].fillna(wage_data['average_wage'].mean(), inplace=True)
wage_data.isnull().sum()

In [ ]:
wage_dupl = wage_data[wage_data.duplicated()]
print('Duplicate rows: ', wage_dupl)

In [ ]:
# join df on tract
main_df = pd.merge(clean_unemploymentDf, wage_data, on=['tract', 'uu_id'], how='outer')

In [ ]:
main_df

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query_main = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
query_job = bigquery_client.query(query_main)
unemployment_data = query_job.to_dataframe()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [ ]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()

In [ ]:
query_pred = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
query_job = bigquery_client.query(query_pred)
prediction_list = query_job.to_dataframe()